# File description
Create the 2 object detection toy datasets: `Shape` and `Object Detection MNIST`. `Shape` is custom made with the help of `opencv`, whereas `Object Detection MNIST` was created with [this github repo](https://github.com/hukkelas/MNIST-ObjectDetection)<br>
<br>
Both datasets are illustrated below (shape left and MNIST right)
<table><tr>
<td> <img src="../illustration_images/shape_dataset_example.png" width="400" /> 
<td> <img src="../illustration_images/MNIST_examples.png" width="400" />
</tr></table>

# Imports

In [3]:
import dutils as U
U.jupyter_ipython.adjust_screen_width(75)

from glob import glob
import numpy as np
import random
import cv2

# Shape
The data was created with the following noteworthy settings:

    1.) 900 train images, 100 test images.
    2.) Shapes: Triangles and circles
    3.) Image resolution is 256x256
    4.) Bounding box annotation are in YOLO-format

## Dataset generator

In [50]:
def get_image_shape(width=256, height=256):
    
    image = np.zeros((width, height, 3)).astype(np.uint8)
    avg_dim = (width+height)/2
    annotations = ""
    
    # Pick center and size
    c = [random.randint(40, 216), random.randint(40, 216)]
    r = random.randint(20,40)
    
    if random.random() > 0.5:
        # Draw rectancle
        t_x = [c[0]-r, c[0],  c[0]+r]
        t_y = [c[1], c[1]-r*2,  c[1]]
        cv2.fillPoly(image, np.array([list(zip(t_x, t_y))]), (180, 119, 31))
        
        # Annotations
        w, h = max(t_x)-min(t_x), max(t_y)-min(t_y)
        x, y = c[0], c[1] - h/2
        annotations = f"0 {min(1.0, x/width)} {min(1.0, y/height)} {w/width} {h/height}"
    
    else:
        # Draw circle and annotations
        cv2.circle(image, c, r, color=(14, 127, 255), thickness=-1)
        annotations += f"1 {c[0]/width} {c[1]/height} {r*2/width} {r*2/height}"
    
    return image, annotations


## Create dataset

In [51]:
images = []
bbs_all = []
gen_num = 1000
for i in range(gen_num):
    image, bbs = get_image_shape()
    images.append(image)
    bbs_clean = [float(bb) for bb in bbs.split(" ")]
    bbs_all.append(bbs_clean)
    
# To disk
for i, (image, bbs) in enumerate(zip(images, bbs_all)):
    dataset_type = "train" if i < int(0.9*gen_num) else "valid"
    U.input_output.make_file(f"./data_shape/{dataset_type}/labels/{i}.txt", allow_override=True)
    bb_text = " ".join(str(bb) for bb in bbs)
    U.input_output.write_to_file(f"./data_shape/{dataset_type}/labels/{i}.txt", bb_text)
    cv2.imwrite(f"./data_shape/{dataset_type}/images/{i}.PNG", image)

## Visualize

In [ ]:
images_with_bb = []
for image, bb in zip(images, bbs_all):
    image_with_bb = image.copy()
    U.pytorch.yolo_draw_single_bb_cv2(image_with_bb, *bb[1:],  color=[200, 0, 0])
    images_with_bb.append(image_with_bb)

# Save a picture for the report
final_image = U.jupyter_ipython.show_image(random.choices(images_with_bb, k=8), image_border=200, return_image=True)
cv2.imwrite("./shape_dataset_example.png", final_image)

# MNIST
The data was created with the following noteworthy settings:

    1.) 1000 train images, 1000 test images.
    2.) between 1 and 3 digits per images.
    3.) Each digit varies from 50 to 100 pixels.
    4.) Image resolution is 256x256
    5.) Bounding box annotation are in YOLO-format

In [ ]:
def xyxy2xywh(bb, img_width=256, img_height=256):
    x1, y1, x2, y2 = bb
    bb_width, bb_height = (x2 - x1), (y2 - y1)
    w, h = bb_width/img_width, bb_height/img_width
    x = (x1 + bb_width / 2) / img_width
    y = (y1 + bb_height / 2) / img_height
    return [x,y,w,h]

image_paths = glob("C:/Users/JK/Desktop/Onedrive/Bachelor/code_jupyter/mnist_detection/train/images/*.png")[:8]
label_paths = glob("C:/Users/JK/Desktop/Onedrive/Bachelor/code_jupyter/mnist_detection/train/labels//*.txt")[:8]

images = []
for image_path, label_path in zip(image_paths, label_paths):
    image = U.images.load_image(image_path, "rgb")
    labels_text = U.input_output.read_file(label_path).split("\n")[1:-1]
    labels_xyxy = [list(map(int, l.split(",")[1:])) for l in labels_text]
    labels_xywh = [xyxy2xywh(l) for l in labels_xyxy]
    
    for bb in labels_xywh:
        U.pytorch.yolo_draw_single_bb_cv2(image, *bb, color=(200,0,0))
    images.append(image)

# Save a picture for the report
final_image = U.jupyter_ipython.show_image(images, image_border=200, return_image=True)
cv2.imwrite("./MNIST_examples.png", final_image)